In [13]:
import os, json, pickle, csv, pandas
import numpy as np
from classDeclarations import file_data
from random import sample
MAX_SAMPLE_SIZE = 100
BYTE_SIZE = 262144

In [2]:
file_extensions = []
with open("../file_extensions.txt", "r") as fp_ext:
    file_extensions = [line.rstrip() for line in fp_ext]


file_extensions = [''] + [x.lower() for x in file_extensions]
print(file_extensions)

['', '0', '23', 'csv', 'doc', 'ds_store', 'otl', 'pdf', 'txt', 'temporaryitems', 'xls', 'zip', '_hydrochem_data', '_pacificasource', '_readme', '_crossover_plots', 'apdisk', 'avi', 'bmp', 'cdf', 'cfg', 'cfl', 'cid', 'csv', 'csv~', 'dat', 'db', 'dbf', 'des', 'dob', 'doc', 'docx', 'exe', 'fig', 'final', 'for', 'geos', 'gif', 'gmtcommands4', 'gz', 'hob', 'htm', 'html', 'idv', 'info', 'inp', 'inv', 'jnl', 'jpg', 'kmz', 'log', 'lst', 'm', 'mat', 'mp4', 'm~', 'nas', 'nc', 'note', 'pdf', 'png', 'ppt', 'prj', 'prn', 'ps', 'py', 'qc', 'rtf', 'save', 'sec', 'shp', 'shx', 'sta', 'tas', 'tsv', 'txt', 'txt~', 'uni', 'var', 'xls', 'xlsx', 'xml']


In [3]:
Path = "../../CDIACPub8"

imgs = []
tabular = []
freetext = []
json_xml = []
netcdf = []
unknown = []

missing_count = 0
df = pandas.read_csv('../CDIACFileData/labels/cdiac_naivetruth_processed.csv')


labelSeriesObj = df['file_label']
pathSeriesObj = df['path']


for i in range(labelSeriesObj.size):
    if labelSeriesObj[i] == 'image':
        imgs.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'tabular':
        tabular.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'freetext':
        freetext.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'json/xml':
        json_xml.append(pathSeriesObj[i])
    elif labelSeriesObj[i] == 'netcdf':
        netcdf.append(pathSeriesObj[i])
    else: 
        unknown.append(pathSeriesObj[i])


In [14]:
print("loading files now...")

with open('../CDIACFileData/ByteVectors/byte_vector_dict_{bs}B.pkl'.format(bs=BYTE_SIZE), "rb") as fp1:
    one_gram = pickle.load(fp1)
with open('../CDIACFileData/ByteVectors/byte_vector_dict_{bs}B_2grams.pkl'.format(bs=BYTE_SIZE), "rb") as fp2:
    two_gram = pickle.load(fp2)

print("loading files done!")
#byte_distr = pandas.DataFrame.from_dict(one_gram)
#two_grams_dicts = pandas.DataFrame.from_dict(two_gram)
#best_extractors = json.load(open("../CorrelatingExtractors/best_extractors.json"))

loading files now...
loading files done!


In [15]:
print(len(one_gram))
print(len(two_gram))

print(one_gram['/home/cc/CDIACPub8/nov_2001_f.xls'].shape)
print(two_gram['/home/cc/CDIACPub8/nov_2001_f.xls'].shape)



20427
20427
(262144,)
(262143,)


In [28]:
'''
for filename, vector in one_gram.items():
	name, extension = os.path.splitext(filename) # file_name actually holds file path
	extension = extension[1:].lower()

	one_gram[filename] = np.append(one_gram[filename], file_extensions.index(extension))
	one_gram[filename] = np.append(one_gram[filename], os.path.getsize(filename))

for filename, vector in two_gram.items():
	name, extension = os.path.splitext(filename) # file_name actually holds file path
	extension = extension[1:].lower()

	two_gram[filename] = np.append(two_gram[filename], file_extensions.index(extension))
	two_gram[filename] = np.append(two_gram[filename], os.path.getsize(filename))
'''
	

In [6]:
print(len(one_gram))
print(len(two_gram))

print(one_gram['/home/cc/CDIACPub8/nov_2001_f.xls'].shape)
print(two_gram['/home/cc/CDIACPub8/nov_2001_f.xls'].shape)

    


20427
20427
(512,)
(511,)


In [7]:
def generate_sample(iter):
    img_sample_names = sample(imgs, MAX_SAMPLE_SIZE)
    tabular_sample_names = sample(tabular, MAX_SAMPLE_SIZE)
    freetext_sample_names = sample(freetext, MAX_SAMPLE_SIZE)
    json_xml_sample_names = sample(json_xml, MAX_SAMPLE_SIZE)
    netcdf_sample_names = sample(netcdf, MAX_SAMPLE_SIZE)
    unknown_sample_names = sample(unknown, MAX_SAMPLE_SIZE)

    img_data = []
    tabular_data = []
    freetext_data = []
    json_xml_data = []
    netcdf_data = []
    unknown_data = []
    dataset = [img_data, tabular_data, freetext_data, json_xml_data, netcdf_data, unknown_data]

    for idx, data_list in enumerate(dataset):
        curr_sample = []
        if idx == 0:
            best_extractor = 'image'
            curr_sample = img_sample_names
        elif idx == 1:
            best_extractor = 'tabular'
            curr_sample = tabular_sample_names
        elif idx == 2:
            best_extractor = 'freetext'
            curr_sample = freetext_sample_names
        elif idx == 3:
            best_extractor = 'json/xml'
            curr_sample = json_xml_sample_names
        elif idx == 4:
            best_extractor = 'netcdf'
            curr_sample = netcdf_sample_names
        elif idx == 5:
            best_extractor = 'unknown'
            curr_sample = unknown_sample_names
        print("Done with index", idx)

        for file_name in curr_sample:
            curr_file_data = file_data(file_name, one_gram[file_name], two_gram[file_name].astype(np.uint16), best_extractor)
            data_list.append(curr_file_data)

        print('Dumping!')

        with open('gathered_data_byte_vector_raw/sample{x}/byte_vectors_v{x}_{bs}B.pkl'.format(x=iter, bs=BYTE_SIZE), 'wb+') as handle:
            pickle.dump(dataset, file=handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('Dumped.')

In [16]:
for i in range(1, 4):
	generate_sample(i)
	print("Generated:", i)


Done with index 0
Dumping!
Dumped.
Done with index 1
Dumping!
Dumped.
Done with index 2
Dumping!
Dumped.
Done with index 3
Dumping!
Dumped.
Done with index 4
Dumping!
Dumped.
Done with index 5
Dumping!
Dumped.
Generated: 1
Done with index 0
Dumping!
Dumped.
Done with index 1
Dumping!
Dumped.
Done with index 2
Dumping!
Dumped.
Done with index 3
Dumping!
Dumped.
Done with index 4
Dumping!
Dumped.
Done with index 5
Dumping!
Dumped.
Generated: 2
Done with index 0
Dumping!
Dumped.
Done with index 1
Dumping!
Dumped.
Done with index 2
Dumping!
Dumped.
Done with index 3
Dumping!
Dumped.
Done with index 4
Dumping!
